## Results for Tables 1, 2, 3 for CARLA methods

In [21]:
from carla.data.catalog import OnlineCatalog
import pandas as pd
import os
import numpy as np

# load catalog dataset
data_name = "adult"
data_name = "give_me_some_credit"
# data_name = 'compas'
dataset = OnlineCatalog(data_name)


In [22]:
from carla.models.catalog import MLModelCatalog
import torch
torch.manual_seed(0)

ml_model = MLModelCatalog(
        dataset, 
        model_type="ann", 
        load_online=False, 
        backend="pytorch"
    )


if data_name == 'adult':
    ml_model.train(
    learning_rate=0.002,
    epochs=20,
    batch_size=1024,
    hidden_size=[18, 9, 3],
    force_train=True, # don't forget to add this or it might load an older model from disk
    )
elif data_name == 'give_me_some_credit':
    ml_model.train(
    learning_rate=0.002,
    epochs=20,
    batch_size=2048,
    hidden_size=[18, 9, 3],
    force_train=True, # don't forget to add this or it might load an older model from disk
    )
elif data_name == 'compas':
    ml_model.train(
    learning_rate=0.002,
    epochs=25,
    batch_size=25,
    hidden_size=[18, 9, 3],
    force_train=True, # don't forget to add this or it might load an older model from disk
    )


balance on test set 0.9320676322926885, balance on test set 0.9323107818018143
Epoch 0/19
----------


/nr/samba/user/anr/anaconda3/envs/carla_github/lib/python3.7/site-packages/carla/models/catalog/ANN_TORCH/model_ann.py:56: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  x = self.softmax(x)


train Loss: 0.2694 Acc: 0.9321

test Loss: 0.2232 Acc: 0.9323

Epoch 1/19
----------
train Loss: 0.2133 Acc: 0.9321

test Loss: 0.2077 Acc: 0.9323

Epoch 2/19
----------
train Loss: 0.2024 Acc: 0.9321

test Loss: 0.1995 Acc: 0.9330

Epoch 3/19
----------
train Loss: 0.1978 Acc: 0.9337

test Loss: 0.1992 Acc: 0.9344

Epoch 4/19
----------
train Loss: 0.1968 Acc: 0.9349

test Loss: 0.2000 Acc: 0.9346

Epoch 5/19
----------
train Loss: 0.1960 Acc: 0.9350

test Loss: 0.2025 Acc: 0.9350

Epoch 6/19
----------
train Loss: 0.1954 Acc: 0.9346

test Loss: 0.1959 Acc: 0.9353

Epoch 7/19
----------
train Loss: 0.1950 Acc: 0.9349

test Loss: 0.1961 Acc: 0.9351

Epoch 8/19
----------
train Loss: 0.1946 Acc: 0.9351

test Loss: 0.1965 Acc: 0.9355

Epoch 9/19
----------
train Loss: 0.1945 Acc: 0.9352

test Loss: 0.1952 Acc: 0.9355

Epoch 10/19
----------
train Loss: 0.1945 Acc: 0.9349

test Loss: 0.1954 Acc: 0.9356

Epoch 11/19
----------
train Loss: 0.1944 Acc: 0.9349

test Loss: 0.1965 Acc: 0.9356



In [23]:
from sklearn import metrics

if data_name == 'adult':
    y = dataset.df_test['income']
elif data_name == 'give_me_some_credit':
    y = dataset.df_test['SeriousDlqin2yrs']
elif data_name == 'compas':
    y = dataset.df_test['score']

pred = ml_model.predict_proba(dataset.df_test)
pred = [row[1] for row in pred]
fpr, tpr, thresholds = metrics.roc_curve(y, pred, pos_label=1)
metrics.auc(fpr, tpr)

0.8172017419076699

In [24]:
from carla.models.negative_instances import predict_negative_instances
import carla.recourse_methods.catalog as recourse_catalog

factuals = predict_negative_instances(ml_model, dataset.df)
test_factual = factuals.iloc[:100]

In [25]:
dataset.inverse_transform(test_factual).loc[263]

RevolvingUtilizationOfUnsecuredLines       1.002647
age                                       38.000000
NumberOfTime30-59DaysPastDueNotWorse       2.000000
DebtRatio                                  0.472543
MonthlyIncome                           3550.000000
NumberOfOpenCreditLinesAndLoans            8.000000
NumberOfTimes90DaysLate                    1.000000
NumberRealEstateLoansOrLines               0.000000
NumberOfTime60-89DaysPastDueNotWorse       1.000000
NumberOfDependents                         4.000000
SeriousDlqin2yrs                           1.000000
Name: 263, dtype: float64

In [26]:
# ml_model.predict_proba(dataset.df)[106]

## Violation

In [27]:
def intersection(lst1, lst2):
    return list(set(lst1) & set(lst2))

results = []
for method in ['cchvae', 'cem-vae', 'revise', 'clue', 'crud', 'face']:
    if data_name == 'adult':
        cfs = pd.read_csv("Results/adult_manifold_results.csv")
    elif data_name == 'give_me_some_credit':
        cfs = pd.read_csv("Results/give_me_some_credit_manifold_results.csv")
    elif data_name == 'compas':
        cfs = pd.read_csv("Results/compas_manifold_results.csv")
    factuals = predict_negative_instances(ml_model, dataset.df)
    test_factual = factuals.iloc[:100]

    cfs.rename(columns={'Unnamed: 0': 'index'}, inplace=True)
    cfs.set_index(['index'], inplace=True)

    df_cfs = cfs[cfs['method'] == method].drop(['method',	'data'], axis=1)

    nan_idx = df_cfs.index[df_cfs.isnull().any(axis=1)]

    output_factuals = test_factual.copy()
    output_counterfactuals = df_cfs.copy()

    output_factuals = output_factuals.drop(index=nan_idx)
    output_counterfactuals = output_counterfactuals.drop(index=nan_idx)

    test_factual = output_factuals
    df_cfs = output_counterfactuals

    df_decoded_cfs = dataset.inverse_transform(df_cfs.copy())

    df_factuals = dataset.inverse_transform(test_factual.copy())

    # print(df_decoded_cfs)
    # print(df_factuals)
    # check continuous using np.isclose to allow for very small numerical differences
    cfs_continuous_immutable = df_decoded_cfs[
        intersection(dataset.continuous, dataset.immutables)
    ]
    factual_continuous_immutable = df_factuals[
        intersection(dataset.continuous, dataset.immutables)
    ]

    continuous_violations = np.invert(
        np.isclose(cfs_continuous_immutable, factual_continuous_immutable)
    )
    continuous_violations = np.sum(continuous_violations, axis=1).reshape(
        (-1, 1)
    )  # sum over features

    # check categorical by boolean comparison
    cfs_categorical_immutable = df_decoded_cfs[
        intersection(dataset.categorical, dataset.immutables)
    ]
    factual_categorical_immutable = df_factuals[
        intersection(dataset.categorical, dataset.immutables)
    ]

    categorical_violations = cfs_categorical_immutable != factual_categorical_immutable
    categorical_violations = np.sum(categorical_violations.values, axis=1).reshape(
        (-1, 1)
    )  # sum over features

    total_violations = continuous_violations + categorical_violations

    for x in total_violations:
        results.append(x[0])
        
final_results = cfs.copy()
final_results.dropna(inplace=True)
final_results['violations'] = results

/nr/samba/user/anr/anaconda3/envs/carla_github/lib/python3.7/site-packages/carla/models/catalog/ANN_TORCH/model_ann.py:56: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  x = self.softmax(x)
/nr/samba/user/anr/anaconda3/envs/carla_github/lib/python3.7/site-packages/carla/models/catalog/ANN_TORCH/model_ann.py:56: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  x = self.softmax(x)
/nr/samba/user/anr/anaconda3/envs/carla_github/lib/python3.7/site-packages/carla/models/catalog/ANN_TORCH/model_ann.py:56: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  x = self.softmax(x)
/nr/samba/user/anr/anaconda3/envs/carla_github/lib/python3.7/site-packages/carla/models/catalog/ANN_TORCH/model_ann.py:56: UserWarning: Implicit dimension choice for softmax has been deprecated. Change 

## Distance

In [28]:
def intersection(lst1, lst2):
    return list(set(lst1) & set(lst2))

results = []
for method in ['cchvae', 'cem-vae', 'revise', 'clue', 'crud', 'face']:
    if data_name == 'adult':
        cfs = pd.read_csv("Results/adult_manifold_results.csv")
    elif data_name == 'give_me_some_credit':
        cfs = pd.read_csv("Results/give_me_some_credit_manifold_results.csv")
    elif data_name == 'compas':
        cfs = pd.read_csv("Results/compas_manifold_results.csv")
    factuals = predict_negative_instances(ml_model, dataset.df)
    test_factual = factuals.iloc[:100]

    cfs.rename(columns={'Unnamed: 0': 'index'}, inplace=True)
    cfs.set_index(['index'], inplace=True)

    df_cfs = cfs[cfs['method'] == method].drop(['method',	'data'], axis=1)

    nan_idx = df_cfs.index[df_cfs.isnull().any(axis=1)]

    output_factuals = test_factual.copy()
    output_counterfactuals = df_cfs.copy()

    output_factuals = output_factuals.drop(index=nan_idx)
    output_counterfactuals = output_counterfactuals.drop(index=nan_idx)

    factual_without_nans = output_factuals
    counterfactuals_without_nans = output_counterfactuals

    # results.append(total_violations.sum())

    columns = ["Distance_1", "Distance_2", "Distance_3", "Distance_4"]

    
    arr_f = ml_model.get_ordered_features(factual_without_nans).to_numpy()
    arr_cf = ml_model.get_ordered_features(
        counterfactuals_without_nans
    ).to_numpy()

    delta = arr_f - arr_cf 

    d1 = np.sum(np.invert(np.isclose(delta, np.zeros_like(delta))), axis=1, dtype=np.float).tolist()
    d1_old = np.sum(delta.round(2) != 0, axis=1, dtype=np.float).tolist()

    d2 = np.sum(np.abs(delta), axis=1, dtype=np.float).tolist()
    d3 = np.sum(np.square(np.abs(delta)), axis=1, dtype=np.float).tolist()

    results.append(pd.DataFrame({'L0': d1, 'L1': d2, 'L2': d3, 'time': df_cfs['time (seconds)'].mean()}))

temp = pd.concat(results)
temp.index = final_results.index
final_results = pd.concat([final_results, temp], axis=1)


/nr/samba/user/anr/anaconda3/envs/carla_github/lib/python3.7/site-packages/carla/models/catalog/ANN_TORCH/model_ann.py:56: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  x = self.softmax(x)
/nr/samba/user/anr/anaconda3/envs/carla_github/lib/python3.7/site-packages/carla/models/catalog/ANN_TORCH/model_ann.py:56: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  x = self.softmax(x)
/nr/samba/user/anr/anaconda3/envs/carla_github/lib/python3.7/site-packages/carla/models/catalog/ANN_TORCH/model_ann.py:56: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  x = self.softmax(x)
/nr/samba/user/anr/anaconda3/envs/carla_github/lib/python3.7/site-packages/carla/models/catalog/ANN_TORCH/model_ann.py:56: UserWarning: Implicit dimension choice for softmax has been deprecated. Change 

## Validity

In [29]:
results = []
for method in ['cchvae', 'cem-vae', 'revise', 'clue', 'crud', 'face']:
    if data_name == 'adult':
        cfs = pd.read_csv("Results/adult_manifold_results.csv")
        y_col = 'income'
    elif data_name == 'give_me_some_credit':
        cfs = pd.read_csv("Results/give_me_some_credit_manifold_results.csv")
        y_col = "SeriousDlqin2yrs"
    elif data_name == 'compas':
        cfs = pd.read_csv("Results/compas_manifold_results.csv")
        y_col = "score"

    cfs.rename(columns={'Unnamed: 0': 'index'}, inplace=True)
    cfs.set_index(['index'], inplace=True)

    df_cfs = cfs[cfs['method'] == method].drop(['method',	'data'], axis=1)
    results.append(pd.DataFrame({'validity': df_cfs[y_col]}))

In [30]:
temp = ml_model.predict_proba(final_results)
temp2 = []
temp3 = []
for x in temp:
    temp2.append(x[1]>= 0.5) #  >= 0.5

    temp3.append(x[1]) #  >= 0.5
    
final_results['validity'] = temp2
final_results['prediction'] = temp3
final_results['validity'] = final_results['validity'].astype(int)

/nr/samba/user/anr/anaconda3/envs/carla_github/lib/python3.7/site-packages/carla/models/catalog/ANN_TORCH/model_ann.py:56: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  x = self.softmax(x)


## Feasibility

In [31]:
from sklearn.neighbors import NearestNeighbors

results = []
for method in ['cchvae', 'cem-vae', 'revise', 'clue', 'crud', 'face']:
    if data_name == 'adult':
        cfs = pd.read_csv("Results/adult_manifold_results.csv")
    elif data_name == 'give_me_some_credit':
        cfs = pd.read_csv("Results/give_me_some_credit_manifold_results.csv")
    elif data_name == 'compas':
        cfs = pd.read_csv("Results/compas_manifold_results.csv")
    factuals = predict_negative_instances(ml_model, dataset.df)
    test_factual = factuals.iloc[:100]

    cfs.rename(columns={'Unnamed: 0': 'index'}, inplace=True)
    cfs.set_index(['index'], inplace=True)

    df_cfs = cfs[cfs['method'] == method].drop(['method',	'data'], axis=1)

    nan_idx = df_cfs.index[df_cfs.isnull().any(axis=1)]

    output_factuals = test_factual.copy()
    output_counterfactuals = df_cfs.copy()

    output_factuals = output_factuals.drop(index=nan_idx)
    output_counterfactuals = output_counterfactuals.drop(index=nan_idx)

    factual_without_nans = output_factuals
    counterfactuals_without_nans = output_counterfactuals


    cols = dataset.df.columns
    cols.drop(dataset.target)

    nbrs = NearestNeighbors(n_neighbors=5).fit(factual_without_nans[cols].values)

    for i, row in counterfactuals_without_nans[cols].iterrows():
        knn = nbrs.kneighbors(row.values.reshape((1, -1)), 5, return_distance=True)[0]
        
        results.append(np.mean(knn))
final_results['feasibility'] = results

/nr/samba/user/anr/anaconda3/envs/carla_github/lib/python3.7/site-packages/carla/models/catalog/ANN_TORCH/model_ann.py:56: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  x = self.softmax(x)
/nr/samba/user/anr/anaconda3/envs/carla_github/lib/python3.7/site-packages/carla/models/catalog/ANN_TORCH/model_ann.py:56: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  x = self.softmax(x)
/nr/samba/user/anr/anaconda3/envs/carla_github/lib/python3.7/site-packages/carla/models/catalog/ANN_TORCH/model_ann.py:56: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  x = self.softmax(x)
/nr/samba/user/anr/anaconda3/envs/carla_github/lib/python3.7/site-packages/carla/models/catalog/ANN_TORCH/model_ann.py:56: UserWarning: Implicit dimension choice for softmax has been deprecated. Change 

In [50]:
pd.set_option('display.max_columns', None)

temp = final_results[['L0', 'L1', 'L2',  'feasibility', 'violations', 'validity', 'prediction']]#.groupby(['method']).mean()
cfs.dropna(inplace=True)
temp = pd.concat([temp, dataset.inverse_transform(cfs)], axis=1)


In [51]:
import pandas as pd

results = pd.read_csv(f"/nr/samba/user/anr/pkg/MCCE_Python/Results/{data_name}_mcce_results_k_10000_n_100.csv", index_col=0)

results['data'] = data_name
results['method'] = 'mcce'
results.rename(columns={'violation': 'violations'}, inplace=True)

preds = ml_model.predict_proba(results)
new_preds = []
for x in preds:
    new_preds.append(x[1])
results['prediction'] = new_preds
results = dataset.inverse_transform(results)
results.head(1)

results['validity'] = np.where(np.asarray(new_preds) >= 0.5, 1, 0)

# results.loc[263]

/nr/samba/user/anr/anaconda3/envs/carla_github/lib/python3.7/site-packages/carla/models/catalog/ANN_TORCH/model_ann.py:56: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  x = self.softmax(x)


In [52]:
temp = pd.concat([temp, results[temp.columns]])

temp2 = factuals.copy()
preds = ml_model.predict_proba(temp2)
new_preds = []
for x in preds:
    new_preds.append(x[1])
temp2['prediction'] = new_preds
temp2 = dataset.inverse_transform(temp2)
temp2.head(1)
temp2['L0'] = np.nan
temp2['L1'] = np.nan
temp2['L2'] = np.nan
temp2['validity'] = np.nan
temp2['violations'] = np.nan
temp2['feasibility'] = np.nan
temp2['time (seconds)'] = np.nan
temp2['method'] = 'original'
temp2['data'] = data_name

temp = pd.concat([temp, temp2.iloc[0:100][temp.columns]], axis=0)
temp

/nr/samba/user/anr/anaconda3/envs/carla_github/lib/python3.7/site-packages/carla/models/catalog/ANN_TORCH/model_ann.py:56: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  x = self.softmax(x)


,L0,L1,L2,feasibility,violations,validity,prediction,method,data,RevolvingUtilizationOfUnsecuredLines,age,NumberOfTime30-59DaysPastDueNotWorse,DebtRatio,MonthlyIncome,NumberOfOpenCreditLinesAndLoans,NumberOfTimes90DaysLate,NumberRealEstateLoansOrLines,NumberOfTime60-89DaysPastDueNotWorse,NumberOfDependents,SeriousDlqin2yrs,time (seconds)
9,10.0,1.312929,0.305116,0.439664,1.0,1.0,0.958935,cchvae,give_me_some_credit,4.179865,51.240301,0.267901,9.327533,6252.902380,8.485504,0.111866,0.979954,0.086648,0.817739,1.0,202.023006
151,10.0,0.965035,0.171564,0.439572,1.0,1.0,0.958918,cchvae,give_me_some_credit,4.180127,51.234049,0.267835,9.326424,6248.205980,8.485100,0.111863,0.979335,0.086686,0.817736,1.0,202.023006
155,10.0,1.162739,0.338375,0.439668,1.0,1.0,0.958937,cchvae,give_me_some_credit,4.179217,51.240719,0.267876,9.327670,6253.067052,8.485495,0.111866,0.979976,0.086648,0.817747,1.0,202.023006
263,10.0,1.683243,0.780067,0.439565,1.0,1.0,0.958915,cchvae,give_me_some_credit,4.180795,51.233605,0.267863,9.326275,6247.850267,8.485077,0.111863,0.979294,0.086688,0.817727,1.0,202.023006
287,10.0,1.434018,0.292133,0.439559,1.0,1.0,0.958917,cchvae,give_me_some_credit,4.179566,51.233257,0.267796,9.326341,6247.563210,8.485036,0.111863,0.979244,0.086692,0.817744,1.0,202.023006
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5082,NaN,NaN,NaN,NaN,NaN,NaN,0.483145,original,give_me_some_credit,0.858949,32.000000,2.000000,0.157109,3417.000000,2.000000,2.000000,0.000000,0.000000,3.000000,1.0,NaN
5125,NaN,NaN,NaN,NaN,NaN,NaN,0.299242,original,give_me_some_credit,0.000000,28.000000,1.000000,0.424144,4000.000000,8.000000,6.000000,1.000000,1.000000,0.000000,0.0,NaN
5277,NaN,NaN,NaN,NaN,NaN,NaN,0.314160,original,give_me_some_credit,0.955684,44.000000,1.000000,0.660843,4578.000000,7.000000,6.000000,1.000000,0.000000,0.000000,0.0,NaN
5288,NaN,NaN,NaN,NaN,NaN,NaN,0.372365,original,give_me_some_credit,0.991332,48.000000,1.000000,0.281100,5200.000000,7.000000,1.000000,0.000000,5.000000,3.000000,1.0,NaN


In [49]:
# results = pd.read_csv(f"/nr/samba/user/anr/pkg/MCCE_Python/Results/{data_name}_baseline_results_n_100.csv", index_col=0)
# results['data'] = data_name
# results['method'] = 'baseline'
# results.rename(columns={'violation': 'violations'}, inplace=True)

# preds = ml_model.predict_proba(results)
# new_preds = []
# for x in preds:
#     new_preds.append(x[1])
# results['prediction'] = new_preds
# results = dataset.inverse_transform(results)
# results.head(1)

# results['validity'] = np.where(np.asarray(new_preds) >= 0.5, 1, 0)
# results[temp.columns]
# temp = pd.concat([temp, results[temp.columns]])
# temp

In [53]:
if data_name == 'give_me_some_credit':
    cols = ['method', 'data', 'prediction', 'L0', 'L1', 'L2', 'feasibility', 'violations', 'validity', 'time (seconds)'] + temp.columns[9:-1].to_list()
    temp = temp[cols]
elif data_name == 'adult':
    cols = ['method', 'data', 'prediction', 'L0', 'L1', 'L2', 'feasibility', 'violations', 'validity', 'time (seconds)'] + temp.columns[9:16].to_list() + temp.columns[17:].to_list()
    temp = temp[cols]
temp.to_csv(f"Final_results/{data_name}_results_mcce_and_carla_K_10000_n_100.csv")



## To get Table 1

In [37]:
temp = pd.read_csv("Final_results/adult_results_mcce_and_carla_K_10000_n_100.csv", index_col=0)

to_write = temp[['method', 'L0', 'L2', 'feasibility', 'violations', 'validity', 'time (seconds)']].groupby(['method']).mean()

to_write.reset_index(inplace=True)

CE_N = temp.groupby(['method']).size().reset_index().rename(columns={0: 'CE_N'})
to_write = pd.concat([to_write, CE_N.CE_N], axis=1)

# to_write.sort_values(['method'], inplace=True, ascending=False)
to_write = to_write[['method', 'L0', 'L2', 'feasibility', 'violations', 'validity', 'CE_N', 'time (seconds)']]

print(to_write.to_latex(index=False, float_format="%.2f", ))

\begin{tabular}{lrrrrrrr}
\toprule
   method &    L0 &   L2 &  feasibility &  violations &  validity &  CE\_N &  time (seconds) \\
\midrule
 baseline &  3.18 & 0.66 &         0.08 &        0.00 &      1.00 &    93 &           17.06 \\
   cchvae &  9.39 & 2.67 &         0.93 &        1.29 &      1.00 &   100 &           85.69 \\
  cem-vae &  6.52 & 2.19 &         0.98 &        1.04 &      0.52 &   100 &           81.22 \\
     clue & 11.34 & 2.00 &         1.26 &        1.29 &      1.00 &   100 &          354.79 \\
     crud & 10.66 & 2.33 &         1.09 &        1.29 &      1.00 &   100 &         1145.26 \\
     face &  6.96 & 2.72 &         1.03 &        1.36 &      1.00 &   100 &          955.89 \\
     mcce &  3.06 & 0.68 &         0.04 &        0.00 &      1.00 &   100 &          107.51 \\
 original &   NaN &  NaN &          NaN &         NaN &       NaN &   100 &             NaN \\
   revise &  9.66 & 3.22 &         1.06 &        1.39 &      1.00 &    38 &          907.06 \\
\bott

In [38]:
temp = pd.read_csv("Final_results/give_me_some_credit_results_mcce_and_carla_K_10000_n_100.csv", index_col=0)

to_write = temp[['method', 'L0', 'L2', 'feasibility', 'violations', 'validity', 'time (seconds)']].groupby(['method']).mean()

to_write.reset_index(inplace=True)

CE_N = temp.groupby(['method']).size().reset_index().rename(columns={0: 'CE_N'})
to_write = pd.concat([to_write, CE_N.CE_N], axis=1)

# to_write.sort_values(['method'], inplace=True, ascending=False)
to_write = to_write[['method', 'L0', 'L2', 'feasibility', 'violations', 'validity', 'CE_N', 'time (seconds)']]

print(to_write.to_latex(index=False, float_format="%.2f", ))

\begin{tabular}{lrrrrrrr}
\toprule
   method &    L0 &   L2 &  feasibility &  violations &  validity &  CE\_N &  time (seconds) \\
\midrule
 baseline &  4.24 & 0.18 &         0.02 &        0.00 &      1.00 &   100 &          216.89 \\
   cchvae & 10.00 & 0.41 &         0.44 &        1.00 &      1.00 &   100 &          202.02 \\
  cem-vae &  8.43 & 0.58 &         0.42 &        0.93 &      0.96 &   100 &           86.35 \\
     clue & 10.00 & 0.37 &         0.40 &        0.81 &      1.00 &   100 &          341.87 \\
     crud & 10.00 & 0.38 &         0.49 &        1.00 &      1.00 &   100 &         1153.13 \\
     face &  8.51 & 0.60 &         0.46 &        0.98 &      1.00 &   100 &         3052.45 \\
     mcce &  3.48 & 0.18 &         0.03 &        0.00 &      1.00 &   100 &         1242.18 \\
 original &   NaN &  NaN &          NaN &         NaN &       NaN &   100 &             NaN \\
   revise & 10.00 & 1.00 &         0.79 &        1.00 &      1.00 &   100 &          805.43 \\
\bott

## To get Adult examples in table 2

In [39]:
import pandas as pd

temp = pd.read_csv("Final_results/adult_results_mcce_and_carla_K_10000_n_100.csv", index_col=0)

cols = ['Method', 'Pred', 'Age', 'Work Class', 'FNLWGT', 'Educat.', 'Mar. Stat.', 'Relat.', 'Cap. Gain', 'Cap. Loss', 'Hr.', 'Co.']

cols = ['method', 'prediction', 'age', 'workclass', 'fnlwgt', 'education-num', 'marital-status', 'relationship', 'capital-gain', 'capital-loss', 'hours-per-week', 'native-country']

to_write = temp[cols].loc[31]
to_write.columns = cols
# to_write.sort_values(['Method'], inplace=True, ascending=False)

to_write

print(to_write.to_latex(index=False, float_format="%.0f", ))

# print(to_write.Pred.to_latex(index=False, float_format="%.2f", ))


\begin{tabular}{lrrlrrllrrrl}
\toprule
   method &  prediction &  age &    workclass &  fnlwgt &  education-num & marital-status & relationship &  capital-gain &  capital-loss &  hours-per-week & native-country \\
\midrule
   cchvae &           1 &   39 &      Private &  189644 &             10 &        Married &      Husband &          3653 &           172 &              40 &             US \\
  cem-vae &           1 &   23 &  Non-Private &  190709 &             12 &    Non-Married &  Non-Husband &         10303 &             0 &              52 &             US \\
     clue &           1 &   11 &      Private &  398962 &             10 &    Non-Married &  Non-Husband &         10725 &           -62 &              49 &             US \\
     crud &           1 &   50 &      Private &  147878 &             17 &        Married &      Husband &          8414 &           -30 &              41 &             US \\
     face &           1 &   35 &      Private &   38948 &             11 &   

## To get GMC examples in table 3

In [54]:
temp = pd.read_csv("Final_results/give_me_some_credit_results_mcce_and_carla_K_10000_n_100.csv", index_col=0)
temp.loc[263]

,method,data,prediction,L0,L1,L2,feasibility,violations,validity,time (seconds),RevolvingUtilizationOfUnsecuredLines,age,NumberOfTime30-59DaysPastDueNotWorse,DebtRatio,MonthlyIncome,NumberOfOpenCreditLinesAndLoans,NumberOfTimes90DaysLate,NumberRealEstateLoansOrLines,NumberOfTime60-89DaysPastDueNotWorse,NumberOfDependents,SeriousDlqin2yrs
263,cchvae,give_me_some_credit,0.958915,10.0,1.683243,0.780067,0.439565,1.0,1.0,202.023006,4.180795,51.233605,0.267863,9.326275,6247.850267,8.485077,0.111863,0.979294,0.086688,0.817727,1.000000
263,cem-vae,give_me_some_credit,0.601860,8.0,1.745561,1.213027,0.381716,1.0,1.0,86.349849,0.000000,69.000000,2.000000,0.193935,3000.000061,10.000000,0.000000,0.000000,2.000000,0.000000,1.000000
263,clue,give_me_some_credit,0.996728,10.0,1.376965,0.746909,0.374789,1.0,1.0,341.871374,-2.746603,37.309792,1.986658,-148.846837,627.710725,8.414357,-1.905691,0.269575,-1.185221,2.137450,1.000000
263,crud,give_me_some_credit,0.996066,10.0,1.303359,0.397480,0.566557,1.0,1.0,1153.126735,0.017851,82.796800,0.521500,88.250007,9245.434530,17.240654,-1.007147,1.144465,0.266708,2.025982,1.000000
263,revise,give_me_some_credit,0.952792,10.0,2.401860,0.938449,0.782334,1.0,1.0,805.431304,0.972036,37.999129,0.155846,10.144193,3968.102287,7.551169,0.026027,0.498095,0.080697,0.715548,1.000000
263,face,give_me_some_credit,0.520177,7.0,0.908533,0.322813,0.319428,1.0,1.0,3052.451393,0.000000,42.000000,2.000000,0.460208,2600.000000,6.000000,1.000000,1.000000,1.000000,2.000000,1.000000
263,mcce,give_me_some_credit,0.583052,4.0,0.627327,0.265629,0.034067,0.0,1.0,1242.184716,0.763765,38.000000,2.000000,0.463105,3650.000000,8.000000,1.000000,0.000000,0.000000,2.000000,0.583052
263,original,give_me_some_credit,0.495998,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.002647,38.000000,2.000000,0.472543,3550.000000,8.000000,1.000000,0.000000,1.000000,4.000000,1.000000


In [59]:
cols = ['method', 'prediction', 'age', 'RevolvingUtilizationOfUnsecuredLines', 'NumberOfTime30-59DaysPastDueNotWorse', 'DebtRatio', 'MonthlyIncome', 'NumberOfOpenCreditLinesAndLoans', 'NumberOfTimes90DaysLate', 'NumberRealEstateLoansOrLines', 'NumberOfTime60-89DaysPastDueNotWorse', 'NumberOfDependents']

to_write = temp[cols].loc[263]

cols = ['Method', 'Pred', 'Age', 'Unsec. Lines', 'Nb Days Past 30', 'Debt Ratio', 'Month Inc.', 'Nb Credit Lines', 'Nb Times 90 Days Late', 'Nb Real Estate Loans', 'Nb Times 60 Days Past', 'Nb Dep.']

to_write.columns = cols
# to_write.sort_values(['Method'], inplace=True, ascending=False)

# print(to_write.to_latex(index=False, float_format="%.0f", ))

print(to_write.to_latex(index=False, float_format="%.2f", ))

\begin{tabular}{lrrrrrrrrrrr}
\toprule
   Method &  Pred &   Age &  Unsec. Lines &  Nb Days Past 30 &  Debt Ratio &  Month Inc. &  Nb Credit Lines &  Nb Times 90 Days Late &  Nb Real Estate Loans &  Nb Times 60 Days Past &  Nb Dep. \\
\midrule
   cchvae &  0.96 & 51.23 &          4.18 &             0.27 &        9.33 &     6247.85 &             8.49 &                   0.11 &                  0.98 &                   0.09 &     0.82 \\
  cem-vae &  0.60 & 69.00 &          0.00 &             2.00 &        0.19 &     3000.00 &            10.00 &                   0.00 &                  0.00 &                   2.00 &     0.00 \\
     clue &  1.00 & 37.31 &         -2.75 &             1.99 &     -148.85 &      627.71 &             8.41 &                  -1.91 &                  0.27 &                  -1.19 &     2.14 \\
     crud &  1.00 & 82.80 &          0.02 &             0.52 &       88.25 &     9245.43 &            17.24 &                  -1.01 &                  1.14 &          

In [42]:
features = [ 'age', 'RevolvingUtilizationOfUnsecuredLines', 'NumberOfTime30-59DaysPastDueNotWorse','DebtRatio', 'MonthlyIncome', 'NumberOfOpenCreditLinesAndLoans', 'NumberOfTimes90DaysLate', 'NumberRealEstateLoansOrLines', 'NumberOfTime60-89DaysPastDueNotWorse', 'NumberOfDependents']
metric_names = ['method', 'L0', 'L1', 'violations', 'validity', 'prediction']

temp = dataset.inverse_transform(final_results.dropna()[features])
temp = pd.concat([final_results[metric_names], temp], axis=1)
# temp.sort_values(temp.index.name)

In [43]:
mcce_results = pd.read_csv(f"/nr/samba/user/anr/pkg/MCCE_Python/give_me_some_credit_mcce_results_k_10000.csv")
mcce_results.rename(columns={'Unnamed: 0': 'index'}, inplace=True)
mcce_results.set_index(['index'], inplace=True)

predictions = ml_model.predict_proba(mcce_results)
temp3 = []
for x in predictions:
    temp3.append(x[1]) #  >= 0.5
    
# temp.index = final_results.index
mcce_results['prediction'] = temp3


FileNotFoundError: [Errno 2] No such file or directory: '/nr/samba/user/anr/pkg/MCCE_Python/give_me_some_credit_mcce_results_k_10000.csv'

In [ ]:
mcce_results.sort_values(mcce_results.index.name, inplace=True)
mcce_results['method'] = 'mcce'
mcce_results.rename(columns={'success': 'validity', 'violation': 'violations'}, inplace=True)
temp_mcce = dataset.inverse_transform(mcce_results.dropna()[features])
temp_mcce = pd.concat([mcce_results[metric_names], temp_mcce], axis=1)
# temp_mcce.sort_values(temp_mcce.index.name)

In [ ]:
temp.index

In [ ]:
pd.set_option('display.max_columns', None)
temp2 = pd.concat([temp, temp_mcce], axis=0)

temp2.sort_values(temp2.index.name)

features = ['method', 'prediction', 'age', 'RevolvingUtilizationOfUnsecuredLines', 'NumberOfTime30-59DaysPastDueNotWorse','DebtRatio', 'MonthlyIncome', 'NumberOfOpenCreditLinesAndLoans', 'NumberOfTimes90DaysLate', 'NumberRealEstateLoansOrLines', 'NumberOfTime60-89DaysPastDueNotWorse', 'NumberOfDependents']

temp2.loc[263][features]

In [ ]:
factuals = predict_negative_instances(ml_model, dataset.df)
dataset.inverse_transform(factuals.iloc[3:4])[['age', 'RevolvingUtilizationOfUnsecuredLines', 'NumberOfTime30-59DaysPastDueNotWorse','DebtRatio', 'MonthlyIncome', 'NumberOfOpenCreditLinesAndLoans', 'NumberOfTimes90DaysLate', 'NumberRealEstateLoansOrLines', 'NumberOfTime60-89DaysPastDueNotWorse', 'NumberOfDependents']]

ml_model.predict_proba(factuals.iloc[3:4])
